In [1]:
# Imports #

import numpy as np
import pandas as pd 
import math as m 
import os
import sys
import re
from astropy.utils import data
from astropy.wcs import WCS
from astropy.io import fits
from spectral_cube import SpectralCube
from astropy import units as u
from astropy.modeling import models, fitting
from astropy.io import ascii
from astropy.table import Table

# Set path to your home directory
path='/Users/jane/Desktop'


### Tkin data retrieval ###
# Run once for all molecules- same temperature map for HCO+ and HCN

filename = 's138_temp_regrid.fits'
outfile = 's138_temp_regrid.txt'

# load file, read data, wcs, and header
datfile = fits.open(path+'/'+filename)
dat = datfile[0].data
wcs = WCS(datfile[0].header)

#  y (vert) = 1st position (starting at bottom), x (hor) = 2nd position (starting from left)
xPixT = []
yPixT = []
Tk = []
glatT = []
glongT = []

i = 0
j = 0

# scan through y-axis
for j in range(dat.shape[0]):
    # scan through x-axis 
    for i in range(dat.shape[1]):
        # set the value of the pixel
        zpix = dat[j,i]  # the flux value is found at pixel x, y 
        xPixT.append(i)
        yPixT.append(j)
        Tk.append(zpix)
        gxT, gyT = wcs.wcs_pix2world(i , j , 0) # convert the pixel numbers to WCS coordinates
        glongT.append(gxT)
        glatT.append(gyT)
        

# Set the header line, the parameters, and the format of the parameters to 
# write to an ascii table file        
out = Table()
out['Pix_x'] = xPixT
out['Pix_y'] = yPixT
out['GLat(deg)'] = glatT
out['GLat(deg)'].info.format = '8.5f'
out['GLong(deg)'] = glongT
out['GLong(deg)'].info.format = '8.5f'
out['T(K)'] = Tk
out['T(K)'].info.format = '6.2f'

# Write the values of all pixels to a text file 
ascii.write(out, outfile, overwrite=True, format='tab')
      
# Convert lists to numpy arrays
xPixT = np.array(xPixT)
yPixT = np.array(yPixT)
Tk = np.array(Tk)
      
M = pd.read_csv(path+'/'+outfile, sep="\t", header=None)
df = pd.DataFrame(data=M)

# put pixels in list and remove header (T denotes full list of all pixels from temperature map)
xPixT = np.array(df[0].tolist())
xPixT = xPixT[1:]
yPixT = np.array(df[1].tolist())
yPixT = yPixT[1:]
Tk = (df[4].tolist())
Tk = Tk[1:]

### Radex column density ###
# The column density will be found using Radex for the specified molecules.

# Molecule A
filename='s138_HCO+_gaussfits.txt'       # input fits file name
#outfile = 'radexHCO+.inp' # file name of the radex file
X = pd.read_csv(path+'/'+filename, sep="\t", header=None)
df = pd.DataFrame(data=X)

# put pixel x, y, glat, glong, Tmb, FWHM and VLSR in lists and remove headers
xPixA = np.array(df[0].tolist())
xPixA = xPixA[1:]
yPixA = np.array(df[1].tolist())
yPixA = yPixA[1:]
glatA = np.array(df[2].tolist())
glatA = glatA[1:]
glongA = np.array(df[3].tolist())
glongA = glongA[1:]
tmbA = (df[5].tolist())
tmbA = tmbA[1:]
fwhmA = np.array(df[8].tolist())
fwhmA = fwhmA[1:]
vlsrA = np.array(df[6].tolist())
vlsrA = vlsrA[1:]


# Molecule B
filename='s138_HCN_gaussfits.txt'       # input fits file name
outfile = 'radexHCN.inp' # file name of the radex file
X = pd.read_csv(path+'/'+filename, sep="\t", header=None)
df = pd.DataFrame(data=X)

# put pixel x, y, glat, glong, Tmb, FWHM and VLSR in lists and remove headers
xPixB = np.array(df[0].tolist())
xPixB = xPixB[1:]
yPixB = np.array(df[1].tolist())
yPixB = yPixB[1:]
glatB = np.array(df[2].tolist())
glatB = glatB[1:]
glongB = np.array(df[3].tolist())
glongB = glongB[1:]
tmbB = (df[5].tolist())
tmbB = tmbB[1:]
fwhmB = np.array(df[8].tolist())
fwhmB = fwhmB[1:]
vlsrB = np.array(df[6].tolist())
vlsrB = vlsrB[1:]

# radexcolden(xPix, yPix, glong, glat, tmb, fwhm, vlsr, 'mole', 'radex inp file', 'radex out file', 'column density file')

def radexcolden(xPix, yPix, glong, glat, tmb, fwhm, vlsr, mole, inputFile, outputFile, coldenFile):
    # only keep the pixels from the temperature map data that we have Gaussian fit data for
    tk, xx, yy= [], [], []
    for i in range(len(xPix)):
        for j in range(len(xPixT)):
            if ((xPix[i] == xPixT[j]) and (yPix[i] == yPixT[j])):
                tk.append(Tk[j])
                xx.append(xPixT[j])
                yy.append(yPixT[j])

    if (np.all(xPix != xx) or np.all(yPix != yy)):
        print('error')

    # Run a series of Radex models to retrieve the column density
    maxiter = 100
    debug   = False

    # lists for all new data (omitting null rows)
    xPixFull, yPixFull, glongFull, glatFull, colDen, tkFull, tmbFull, vlsrFull, fwhmFull = [], [], [], [], [], [], [], [], []

    # loop through all pixels

    ran = len(xx)

    for i in range(ran):

        #only run non-zero rows
        if tmb[i] != '0.00':

            #converting values to floats 
            b = np.asarray(tmb[i], dtype=float)
            c = np.asarray(fwhm[i], dtype=float)
            d = np.asarray(tk[i], dtype=float)

            freq = 267.6     # frequency GHz
            tkin = d      # Tkin (K)
            nh2 = 1.0e5      # nH2 cm^-3
            tbg = 2.73       # Tbg (K)
            obs = b          # Observed line intensity (K)- Tmb
            dv = c           # FWHM line width km/s
            bw = 0.01       # Bandwidth (GHz)
            tol = 0.01       # tolerance

            radexpath = '/Users/jane/Desktop/Radex/data/'
            extension = '.dat'

            def write_input(cdmol):
                file = open(path+'/'+inputFile,'w')
                file.write(mole+'.dat\n') 
                file.write(path+'/'+outputFile+'\n')
                file.write(str(freq*(1-bw))+' '+str(freq/(1-bw))+'\n')
                file.write(str(tkin)+'\n')
                file.write('1\n')
                file.write('H2\n')
                file.write(str(nh2)+'\n')
                file.write(str(tbg)+'\n')
                file.write(str(cdmol)+'\n')
                file.write(str(dv)+'\n')
                file.write('0\n')
                file.close()

            def read_radex():
                file  = open(path+'/'+outputFile)
                lines = file.readlines()
                file.close()
                if (lines[-2].split()[-1] != '(erg/cm2/s)'):
                    print("Error: Ambiguous line selection. Reduce bandwidth?")
                    print("See radex.out for details")
                    sys.exit()
                return float(lines[-1].split()[-2])

            # Begin of main program
            oflx = obs*dv
            eps  = 1.0e-20
            iter = 0

            # Starting values of column density and fit residual
            cdmol = 1e12
            ratio = 0

            while (ratio > (1+tol)) or (ratio < (1-tol)) :
                iter += 1
                write_input(cdmol)
                os.system(path+'/Radex/bin/radex <' +path+'/'+inputFile+ '> /dev/null' )
                mflx  = read_radex()
                if (mflx < eps):
                    print("Error: Zero or negative line intensity")
                    print("See radex.out for details")
                    sys.exit()
                if (debug):
                    print("mflx= ",mflx)
                ratio = oflx/mflx
                cdmol = cdmol * ratio
                if (iter > maxiter):
                    print("Maximum number of iterations exceeded")
                    ratio = 1

            #fmt = "CD %7.2e cm^-2"
            colDen.append(cdmol)
            xPixFull.append(xPix[i])
            yPixFull.append(yPix[i])
            glongFull.append(glong[i])
            glatFull.append(glat[i])
            tkFull.append(tk[i])
            tmbFull.append(tmb[i])
            vlsrFull.append(vlsr[i])
            fwhmFull.append(fwhm[i])

    #writing results to table 
    outfile = coldenFile # file name of the gaussian fit results

    out = Table()
    out['Pix_x'] = xPixFull
    out['Pix_y'] = yPixFull
    out['GLat(deg)'] = glatFull
    #out['GLat(deg)'].info.format = '8.7f'
    out['GLong(deg)'] = glongFull
    out['Tk(K)'] = tkFull
    out['Tmb(K)'] = tmbFull
    #out['Tmb(K)'].info.format = '6.2f'
    out['VLSR(km/s)'] = vlsrFull
    #out['VLSR(km/s)'].info.format = '6.2f'
    out['FWHM(km/s)'] = fwhmFull
    #out['FWHM(km/s)'].info.format = '6.2f'
    #out['GLong(deg)'].info.format = '8.7f'
    out['Col. Den. (cm^-2)'] = colDen
    #out['Col. Den. (cm^-2)'].info.format = '6.2f'

    # Write the gaussian fits of all pixels to a text file 
    ascii.write(out, path+'/'+outfile,  overwrite=True, format='tab')

    print('Column density data for', mole, 'saved to', outfile)

radexcolden(xPixA, yPixA, glongA, glatA, tmbA, fwhmA, vlsrA, 'HCO+', 'radexHCO+.inp', 'radexHCO+.out', 'columnDenHCO+.txt')
radexcolden(xPixB, yPixB, glongB, glatB, tmbB, fwhmB, vlsrB, 'HCN', 'radexHCN.inp', 'radexHCN.out', 'columnDenHCN.txt')

Column density data for HCO+ saved to columnDenHCO+.txt
Column density data for HCN saved to columnDenHCN.txt
